In [ ]:
# import sys
import csv
import requests
import json
import pandas as pd
import datetime

today = datetime.date.today().strftime("%y%m%d")
filename = today + "_etf_code.csv"
f = open(filename, "w", encoding="euc-kr", newline="")

url = 'https://finance.naver.com/api/sise/etfItemList.nhn'
json_data = json.loads(requests.get(url).text)
df = pd.json_normalize(json_data['result']['etfItemList'])

df.to_csv(f)
f.close()
# df.to_excel('etf_list_20251030.xlsx')

print("save")


save
